In [1]:
%cd /Users/Deepthi/Documents/Personal/Kaggle/02_BNP Paribas Cardif Claims Management/Data

/Users/Deepthi/Documents/Personal/Kaggle/02_BNP Paribas Cardif Claims Management/Data


In [2]:
import pandas as pd
df = pd.read_csv('train.csv');

In [3]:
import numpy as np
cols = np.array(df.columns)

In [4]:
# Tokenize the data
for colname in cols[1:]:
    dt = df[colname].dtypes
    if dt not in [np.float64, np.int64]:
        df[colname] = pd.factorize(df[colname])[0]

In [5]:
train_labels = np.array(df['target'])

In [6]:
# train_data = np.array(df.drop(['ID'], axis=1))
train_data = np.array(df.drop(['ID','target'], axis=1))

In [7]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(train_data)
train_data = imp.transform(train_data)

In [8]:
from sklearn.cross_validation import train_test_split;
train_set,validation_set,train_label, validation_label = train_test_split(train_data,train_labels, test_size=30000, random_state=4)

In [ ]:
# train_data = train_set[1:]
# train_label = train_set[:1]
# validation_data = validation_set[1:]
# validation_label = validation_set[:1]

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(max_depth=5,n_estimators=15,max_features = 1)
# clf.fit(train_set, train_label)
# clf_probs = clf.predict_proba(validation_set)

In [38]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 300, oob_score = True, random_state=4,max_features = "sqrt",
                              min_samples_leaf = 4,n_jobs=-1)
model.fit(train_set,train_label)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=4,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=-1, oob_score=True, random_state=4,
           verbose=0, warm_start=False)

In [39]:
model.oob_score_

0.15099752225005703

In [40]:
from sklearn.metrics import roc_auc_score
y_oob = model.oob_prediction_
print "c_stat:", roc_auc_score(train_label, y_oob)

c_stat: 0.750554534431


In [ ]:
# 100 and auto - c_stat: 0.726785535289
# 110 - c_stat: 0.727607822327
# 150 - c_stat: 0.730946434986
# 200 and auto - c_stat: 0.734004578315
# 200 and sqrt - c_stat: 0.743173411905 
# 200 and log2 - c_stat: 0.740551554073
# 250 and sqrt - c_stat: 0.745333934385 
# 300 and sqrt - c_stat: 0.746767571391 
# 300 and sqrt , minsampleleaf = 5 - c_stat: 0.749685893455
# 300 and sqrt , minsampleleaf = 4 - c_stat: 0.750554534431

In [41]:
import pandas as pd
df = pd.read_csv('test.csv');
import numpy as np
cols = np.array(df.columns)
# Tokenize the data
for colname in cols[1:]:
    dt = df[colname].dtypes
    if dt not in [np.float64, np.int64]:
        df[colname] = pd.factorize(df[colname])[0]

In [42]:
test_data = np.array(df.drop(['ID'], axis=1))


In [43]:
test_id = np.array(df['ID'])

In [44]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(test_data)
test_set = imp.transform(test_data)

In [45]:
from sklearn.ensemble import RandomForestRegressor

# model = RandomForestRegressor(n_estimators = 100, oob_score = True, random_state=4)
test_output = model.predict(test_set)

In [46]:
np.shape(test_output)

(114393,)

In [47]:
output = pd.DataFrame(zip(test_id,test_output))
output.columns = ['ID','PredictedProb']

In [48]:
output.to_csv('output.csv', index=False)